<a href="https://colab.research.google.com/github/MachineAbiogenesis/MachineLearning/blob/main/News_Classification_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

! pip install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

! kaggle datasets download -d rmisra/news-category-dataset

Saving kaggle.json to kaggle (3).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              237KB  2021-12-03 07:25:55          17228  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01          10422  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           4833  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   2GB  2021-10-22 10:4

In [ ]:

!unzip /content/news-category-dataset.zip -d /content/


Archive:  /content/news-category-dataset.zip
replace /content/News_Category_Dataset_v2.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: /content/News_Category_Dataset_v2.json  


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

df = pd.read_json('/content/News_Category_Dataset_v2.json', lines=True)
df.head()


,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [ ]:
df['headline'][0]


'There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV'

In [ ]:
df['headline']=df['headline']+df['short_description']

In [ ]:
print(len(df[df['headline'] == ""]))

5


In [ ]:
df=df.drop(columns=['link','short_description','date'])

In [ ]:
df_backup = df.copy() #Let's create a copy of the data frame
df_backup

,category,headline,authors
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 5...,Ron Dicker
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker
...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters"
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,


In [ ]:
df.duplicated().sum() 

446

In [ ]:
df.drop_duplicates(keep='last', inplace=True) 

In [ ]:
df.duplicated(subset=['category','headline']).sum()

24

In [ ]:
#remove empty val
print(len(df[df['headline'] == ""]))

5


In [ ]:
df.loc[df['headline'] == "", 'headline'] = np.nan
df.dropna(subset=['headline'], inplace=True)
print(len(df[df['headline'] == ""]))

0


In [ ]:
df['headline']=df['headline']+df['authors']

In [ ]:
#cleaning data
from sklearn.utils import shuffle
df = shuffle(df)
df.reset_index(inplace=True, drop=True) 
len(df)

200402

In [ ]:
def combine_categories(category):
    # THE WORLDPOST becomes WORLD NEWS
    if category == "THE WORLDPOST":
        return "WORLD NEWS"
    # WORLDPOST becomes WORLD NEWS
    if category == "WORLDPOST":
        return "WORLD NEWS"
    # ARTS becomes ARTS & CULTURE
    if category == "ARTS":
        return "ARTS & CULTURE"
    # CULTURE & ARTS becomes ARTS & CULTURE
    if category == "CULTURE & ARTS":
        return "ARTS & CULTURE"
    # HEALTHY LIVING becomes WELLNESS
    if category == "HEALTHY LIVING":
        return "WELLNESS"
    # PARENTING becomes PARENTS
    if category == "PARENTING":
        return "PARENTS"
    # STYLE becomes STYLE & BEAUTY
    if category == "STYLE":
        return "STYLE & BEAUTY"
    # GREEN becomes ENVIRONMENT
    if category == "GREEN":
        return "ENVIRONMENT"
    # TASTE becomes FOOD & DRINK
    if category == "TASTE":
        return "FOOD & DRINK"
    # MONEY becomes BUSINESS
    if category == "MONEY":
        return "BUSINESS"
    return category

df["category"] = df["category"].map(combine_categories)
df['category'] = pd.Categorical(df['category'])

In [ ]:
listarray=df['category'].unique()
sorted(listarray)

['ARTS & CULTURE',
 'BLACK VOICES',
 'BUSINESS',
 'COLLEGE',
 'COMEDY',
 'CRIME',
 'DIVORCE',
 'EDUCATION',
 'ENTERTAINMENT',
 'ENVIRONMENT',
 'FIFTY',
 'FOOD & DRINK',
 'GOOD NEWS',
 'HOME & LIVING',
 'IMPACT',
 'LATINO VOICES',
 'MEDIA',
 'PARENTS',
 'POLITICS',
 'QUEER VOICES',
 'RELIGION',
 'SCIENCE',
 'SPORTS',
 'STYLE & BEAUTY',
 'TECH',
 'TRAVEL',
 'WEDDINGS',
 'WEIRD NEWS',
 'WELLNESS',
 'WOMEN',
 'WORLD NEWS']

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df.category)
df['categorical_label'] = le.transform(df.category)

df.head()


,category,headline,authors,categorical_label
0,BLACK VOICES,Philando Castile's Girlfriend Breaks Down In N...,Willa Frej,1
1,ENTERTAINMENT,Colin Firth Among Few Men To Say They Won't Wo...,Marina Fang,8
2,WELLNESS,"HIV Cure 'Within Months?' Not So Fast, Foundat...",Sara Gates,28
3,POLITICS,Donald Trump Keeps Draining Swamp By Replacing...,Joseph Erbentraut,18
4,PARENTS,Best Coworkers Ever Throw Beyoncé-Themed Showe...,Taylor Pittman,17


In [ ]:
X,Y = df['headline'],df['categorical_label']


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
from keras.preprocessing.text import one_hot
lem=WordNetLemmatizer()

for i,sent in enumerate(X):
  #print(i)
  #sent=re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",sent)

  sent=sent.lower()
  # sent=re.sub('https:\/\/\S+','',sent)
  # sent=re.sub('@\S+','',sent)
  # sent=re.sub('&[a-zA-Z]+;','',sent)
  # sent=re.sub('#\S+','',sent)
  # sent=re.sub('[^a-zA-Z0-9]',' ',sent)
  sent=sent.split()
  wds=''
  sw_nltk =stopwords.words('english')
  for word in sent:
    if word not in sw_nltk:
      word=lem.lemmatize(word)
      wds+=' '
      wds+=word
  X[i]=wds

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:

from sklearn.model_selection import train_test_split
#90% to train , 10% for validation
X_train, X_val, y_train, y_val = train_test_split(X,Y, test_size=0.1, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer=Tokenizer(
    num_words=161393,
    oov_token="<OOV>"
)


tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index

sequences_train = tokenizer.texts_to_sequences(X_train)

padded_seq_train = pad_sequences(sequences_train, maxlen=150,padding='pre',dtype='float32')

sequences_test = tokenizer.texts_to_sequences(X_val)

padded_seq_test = pad_sequences(sequences_test, maxlen=150,padding='pre',dtype='float32')

In [ ]:
len(word_index)

250976

In [ ]:
maxval=0
for i in sequences_train:
  if len(i)>maxval:
    maxval=len(i)
print(maxval) 

156


In [ ]:
print('len of train DF '+str(len(padded_seq_train))+'\n len of Test DF '+str(len(y_train)))

len of train DF 180361
 len of Test DF 180361


In [ ]:
vocab_size = 206893
embedding_dim = 62
max_length = 150

import tensorflow.keras as tk

my_model6 = tk.Sequential([
   tk.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
   tk.layers.Bidirectional(tk.layers.LSTM(64,return_sequences=True)),
   tk.layers.Bidirectional(tk.layers.LSTM(64)),
   tk.layers.Dense(256, activation='relu'),
   tk.layers.Dropout(0.2),
   tk.layers.Dense(31, activation='sigmoid')
])

my_model6.compile(loss=tk.losses.sparse_categorical_crossentropy,
                optimizer=tk.optimizers.Adam(),
                metrics=['acc']
                )
num_epoch = 5

padded_seq_train=np.array(padded_seq_train)
y_train=np.array(y_train)
padded_seq_test=np.array(padded_seq_test)
y_val=np.array(y_val)


history=my_model6.fit(padded_seq_train,
            y_train,
            epochs=num_epoch,
            validation_data=(padded_seq_test, y_val))

                    

Epoch 1/5
5637/5637 [==============================] - 172s 30ms/step - loss: 1.4021 - acc: 0.6159 - val_loss: 0.9340 - val_acc: 0.7441
Epoch 2/5
5637/5637 [==============================] - 166s 29ms/step - loss: 0.7018 - acc: 0.8050 - val_loss: 0.8552 - val_acc: 0.7660
Epoch 3/5
5637/5637 [==============================] - 166s 30ms/step - loss: 0.4230 - acc: 0.8805 - val_loss: 0.9239 - val_acc: 0.7602
Epoch 4/5
5637/5637 [==============================] - 167s 30ms/step - loss: 0.2463 - acc: 0.9294 - val_loss: 1.0516 - val_acc: 0.7552
Epoch 5/5
5637/5637 [==============================] - 167s 30ms/step - loss: 0.1391 - acc: 0.9588 - val_loss: 1.2395 - val_acc: 0.7487
